In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory
from utils.widerface import WIDERDetection, detection_collate


In [2]:
def main(opt):
    torch.manual_seed(opt.seed)
    torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
    Dataset = get_dataset(opt.dataset, opt.task)
    opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
    print(opt)

In [3]:
import re
args=re.split(' +','cspdet --optim adam --multi_scale --exp_id wider_resnet50_csp_multi  --arch cspfpnmulti --batch_size 18 --lr 1e-3 --lr_step 70,80  --gpus 0,1 --num_workers 4 --dataset wider_csp')

In [4]:
print(args)

['cspdet', '--optim', 'adam', '--multi_scale', '--exp_id', 'wider_resnet50_csp_multi', '--arch', 'cspfpnmulti', '--batch_size', '18', '--lr', '1e-3', '--lr_step', '70,80', '--gpus', '0,1', '--num_workers', '4', '--dataset', 'wider_csp']


In [5]:
opt = opts().parse(args=args)
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
print(opt)

logger = Logger(opt)

# os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

print('Creating model...')
model = create_model(opt.arch, opt.heads, opt.head_conv)
# Trainer = train_factory[opt.task]
# trainer = Trainer(opt, model, optimizer)
print('Setting up data...')
val_loader = torch.utils.data.DataLoader(
    Dataset(opt, 'val'),
    batch_size=1,
    shuffle=False,
    num_workers=1,
    pin_memory=False,
    # collate_fn=default_collate
)
train_loader = torch.utils.data.DataLoader(
    Dataset(opt, 'train'),
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.num_workers,
    pin_memory=True,
    drop_last=True,
    # collate_fn=default_collate
)

Fix size testing.
training chunk_sizes: [9, 9]
The output will be saved to  /home/myx/project/face_detection/src/lib/../../exp/cspdet/wider_resnet50_csp_multi
heads {'hm': 1, 'wh': 2, 'off': 2}
Namespace(K=100, aggr_weight=0.0, agnostic_ex=False, arch='cspfpnmulti', aug_ddd=0.5, aug_rot=0, batch_size=18, cat_spec_wh=False, center_thresh=0.1, chunk_sizes=[9, 9], data_dir='/home/myx/project/face_detection/src/lib/../../data', dataset='wider_csp', debug=0, debug_dir='/home/myx/project/face_detection/src/lib/../../exp/cspdet/wider_resnet50_csp_multi/debug', debugger_theme='white', demo='', dense_hp=False, dense_wh=False, dep_weight=1, dim_weight=1, down_ratio=4, eval_oracle_dep=False, eval_oracle_hm=False, eval_oracle_hmhp=False, eval_oracle_hp_offset=False, eval_oracle_kps=False, eval_oracle_offset=False, eval_oracle_wh=False, exp_dir='/home/myx/project/face_detection/src/lib/../../exp/cspdet', exp_id='wider_resnet50_csp_multi', fix_res=True, flip=0.5, flip_test=False, gpus=[0, 1], gpus_s

In [6]:
for i,d in enumerate(val_loader):
    print(i)

index is 0
[[304.37152142   0.         317.01450809   6.07829401]]
index is 1
[[369.29271399 696.6605471  385.09323149 704.        ]]
index is 2
[[686.45342023 502.10598709 704.         547.28326837]
 [699.36121488 679.58816354 704.         704.        ]]
0
1
index is 3
[]
index is 4
[]
index is 5
[]
index is 6
[[362.93674232 515.76650658 704.         704.        ]]
index is 7
2
3
4
5
6
[[  0.         302.35460883   8.15976882 346.39747364]]
index is 8
[]
index is 9
7
8
[]
index is 10
[]
index is 11
[]
index is 12
[]
index is 13
[]
index is 14
[]
index is 15
[[697.09919592 323.29926189 704.         359.177036  ]
 [699.20965322  61.6025566  704.          91.14895881]
 [528.26261186   0.         553.58809947  13.06203868]
 [  0.         321.18880459   9.09011588 361.2874933 ]
 [  0.         139.68947673   9.09011588 173.45679354]
 [  0.          21.50386789   4.86920128  46.82935549]
 [ 30.19468888   0.          51.29926189  10.95158138]
 [243.35087626   0.         264.45544927  10.95158

In [7]:
out=model(d['input'])

/home/myx/miniconda3/envs/torch17/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [6]:
opt = opts().parse(args=args)
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
print(opt)
data=Dataset(opt,'val')

Fix size testing.
training chunk_sizes: [9, 9]
The output will be saved to  /home/myx/project/face_detection/src/lib/../../exp/cspdet/wider_resnet50_csp_multi
heads {'hm': 1, 'wh': 2, 'off': 2}
Namespace(K=100, aggr_weight=0.0, agnostic_ex=False, arch='cspfpnmulti', aug_ddd=0.5, aug_rot=0, batch_size=18, cat_spec_wh=False, center_thresh=0.1, chunk_sizes=[9, 9], data_dir='/home/myx/project/face_detection/src/lib/../../data', dataset='wider_csp', debug=0, debug_dir='/home/myx/project/face_detection/src/lib/../../exp/cspdet/wider_resnet50_csp_multi/debug', debugger_theme='white', demo='', dense_hp=False, dense_wh=False, dep_weight=1, dim_weight=1, down_ratio=4, eval_oracle_dep=False, eval_oracle_hm=False, eval_oracle_hmhp=False, eval_oracle_hp_offset=False, eval_oracle_kps=False, eval_oracle_offset=False, eval_oracle_wh=False, exp_dir='/home/myx/project/face_detection/src/lib/../../exp/cspdet', exp_id='wider_resnet50_csp_multi', fix_res=True, flip=0.5, flip_test=False, gpus=[0, 1], gpus_s

In [9]:
data[0]

index is 0
[[304.37152142   0.         317.01450809   6.07829401]]


{'input': array([[[ 1.5467933 ,  1.1529241 ,  0.7590547 , ..., -0.6622999 ,
          -0.57667613, -0.2855553 ],
         [ 1.6152923 ,  1.2727973 ,  0.9131775 , ..., -0.6622999 ,
          -0.55955136, -0.26843056],
         [ 1.7009162 ,  1.3926706 ,  1.0673003 , ..., -0.6451751 ,
          -0.5424266 , -0.25130582],
         ...,
         [ 0.79330426,  0.810429  ,  0.82755375, ...,  0.31381115,
           0.34806067,  0.38231018],
         [ 0.79330426,  0.810429  ,  0.82755375, ...,  0.33093593,
           0.3651854 ,  0.39943492],
         [ 0.79330426,  0.810429  ,  0.82755375, ...,  0.34806067,
           0.38231018,  0.41655967]],
 
        [[ 1.3081232 ,  0.88795525,  0.48529422, ..., -0.9677871 ,
          -1.0028011 , -0.862745  ],
         [ 1.3606442 ,  0.9754903 ,  0.6078432 , ..., -0.9677871 ,
          -0.9852941 , -0.862745  ],
         [ 1.4131653 ,  1.0630252 ,  0.7303922 , ..., -0.9677871 ,
          -0.9852941 , -0.862745  ],
         ...,
         [ 0.88795525,  

In [10]:
print(out.keys())

dict_keys(['hm', 'wh', 'offset', 'hm_small', 'wh_small', 'offset_small'])


In [11]:
for d in train_loader:
    print(type(d['input']),type(d['hm']))
    input('s')

index is 635index is 680index is 1131index is 11934



[][]

[]
index is 3523
index is 538
[]
index is 2637
[[399.71550146   0.         424.99040883  17.41835632]]
[]
index is 8850
index is 1826
index is 3070
[][]

[[191.8532095  665.08639978 662.17418326 704.        ]]
index is 11393index is 1892

[]
index is 3833
index is 6737
[]
[]
index is 4813
[]index is 1106

index is 11920
[][]

index is 745
index is 2285
[]
index is 5101
[]
[]
[]
index is 7490
index is 7633
[]
index is 7584
index is 10915
[]
[]
index is 6509
index is 12818
[]
index is 1850
[][]

index is 6134
index is 1100
[]
index is 6711
[]
[[692.22815294   5.86926176 704.         133.58857479]][]

[]index is 11672

index is 10843index is 624

index is 11913
[[  0.           0.         100.5556179  102.61442197]
 [147.56199026   0.         169.25723904  15.83342684]
 [689.94320981   0.         704.          14.02548945]]
[]
index is 10811
index is 1716
[]
[]index is 3477

[]
index is 1148index is 1114
[]

[]
index is 2067
[]i

s 


<class 'torch.Tensor'> <class 'torch.Tensor'>
index is 7267
[]
index is 12481
[]
index is 9824
[]
index is 408
[]
index is 8693
[]
index is 12518
[[693.76135906 470.75180143 704.         495.21480933]]
index is 6185
[]
index is 3731
[]
index is 3983
[[  0.         368.38052009   8.38873407 399.43716611]]
index is 10395
[]
index is 1399
[]
index is 6398
[]
index is 10142
[]
index is 1036
[]
index is 10964
[]
index is 9890
[]
index is 8135
[]
index is 5960
[]


KeyboardInterrupt: Interrupted by user

In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import torch
from progress.bar import Bar
from models.data_parallel import DataParallel
from utils.utils import AverageMeter
from trains.cspdet import CspDetLoss

In [10]:
def _get_losses(opt):
    loss_states=['loss']
    loss_states = ['loss', 'hm_loss', 'wh_loss']
    if opt.reg_offset:
        loss_states.append('off_loss')
    if opt.multi_scale:
        # loss_states.append('off_s_loss')
        loss_states.append('hm_s_loss')
        loss_states.append('wh_s_loss')
    loss = CspDetLoss(opt)
    return loss_states, loss

In [11]:
x=_get_losses(opt)

In [ ]:
def _get_losses(self, opt):
    loss_states=['loss']
    loss_states = ['loss', 'hm_loss', 'wh_loss']
    if opt.reg_offset:
        loss_states.append('off_loss')
    if opt.multi_scale:
        # loss_states.append('off_s_loss')
        loss_states.append('hm_s_loss')
        loss_states.append('wh_s_loss')
    loss = CspDetLoss(opt)
    return loss_states, loss